## VGG(Visual Geometry Group)
原始论文：《Very Deep Convolutional Networks For Large-Scale Image Recognition》

论文亮点：
+ 卷积核全部使用的是3 * 3的卷积核
+ 输入图像为224 * 224的RGB图像
+ 预处理：RGB三个通道减去各自通道的均值
+ 每一个卷积层后连接着三个全连接层（Fully-Connected layers，FC）。前两个全连接层各有4096个通道，第三个全连接层是1000种分类。最后一层的激活函数是softmax，其他层的激活函数是ReLU。
+ VGG有六种结构，每一种结构一般以VGG+层数命名。每一种结构具有不同的参数个数。

![VGG](https://cdn-images-1.medium.com/max/800/0*HREIJ1hjF7z4y9Dd.jpg)

+ 训练：使用mini-batch gradient descent with momentum。batch-size是256，momentum是0.9。使用了dropout（radio为0.5）。学习速率初始化为$10^{-2}$。衰减系数为10。
+ 总共训练了370K iterations，74 epochs.
+ 由于层数深的网络很难收敛，因此先训练层数少的网络直至收敛（如configuration A）。然后将A训练好的参数作为更深网络的参数输入进行训练。输入时只将网络A的前四个卷积层和最后三个全连接层作为参数输入，其他的参数仍然进行随机初始化。
+ 随机初始化的均值为0，方差为$10^{-2}$。bias的初始化值为0。

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
def conv(layer_name, x, out_channels, kernel_size=[3,3], stride=[1,1,1,1], is_pretrain=True):
    '''Convolution on wrapper, use ReLU activation after convolution
    Args:
        layer_name: e.g, conv1, poo1, ...
        x: input tensor, [batch_size, height, width, channels]
        out_channels: number of output channels(or convolutional kernels)
        kernel_size: the size of convolutional kernel, VGG paper used: [3, 3]
        stride: A list of ints. 1-D of length 4. VGG paper used: [1, 1, 1, 1]
    Returns:
        4D tensor
    '''